In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import split, when, lit, row_number, udf, col

import random

from pyspark.sql.window import Window

In [ ]:
spark = SparkSession.builder.appName("BeeHive").getOrCreate()

In [ ]:
headers = ['remove', 'Bee ID', 'remove1', 'DaughtersEfficiencyScore', 'remove2', 'Father SIZE', 'Father TYPE', 'remove3', 'X', 'Y', 'Z']
doubles = ['DaughtersEfficiencyScore', 'X', 'Y', 'Z']
integers = ['Father SIZE']

def struct_field(header, doubles, integers):
    
    if header in doubles:
        return StructField(header, DoubleType())
    
    if header in integers:
        return StructField(header, IntegerType())
    
    return StructField(header, StringType())

fields = [struct_field(header, doubles, integers) for header in headers]
schema = StructType(fields)

In [ ]:
df = spark.read.schema(schema).csv('/Users/daniel/dev/duds/pep-data/src/jupyter/dev/test.csv')
df.show(5)

In [ ]:
cols_to_keep = [x for x in df.columns if 'remove' not in x]
df = df.select(*cols_to_keep)

df.show(5)

In [ ]:
df_cleaned = df
df_cleaned = df_cleaned.withColumn('Cycle', split(df_cleaned['Bee ID'], '_').getItem(0))\
                .withColumn('Cycle ID', split(df_cleaned['Bee ID'], '_').getItem(1))

df_cleaned.show(5)

In [ ]:
# cycles = sorted([i[0] for i in df_cleaned.select('Cycle').distinct().collect()])
# print(cycles)
#
# def father_cycle(cycle):
#     cycle_index = cycles.index(cycle)
#     n=3
#     father_cyc = None
#
#     if cycle_index == 0:
#         return father_cyc
#
#     if cycle_index > n:
#         father_cyc = random.randint(cycle_index-n, cycle_index-1)
#
#     elif cycle_index <= n:
#         father_cyc = random.randint(0, cycle_index-1)
#
#     return cycles[father_cyc]

In [ ]:
# from pyspark.sql.functions import udf, col
#
# convertUDF = udf(lambda z: father_cycle(z))
#
# df_cleaned.withColumn("ParentCycle", convertUDF(col('Cycle'))).show()

In [ ]:
w = Window().orderBy('Cycle')
df_cleaned = df_cleaned.withColumn('Continuous ID', row_number().over(w))

df_cleaned.show()

In [ ]:
continuous_min_id_per_cycle = {key : value for key, value  in df_cleaned.groupBy('Cycle').min('Continuous ID').collect()}

continuous_min_id_per_cycle

In [ ]:
cycles = sorted([i[0] for i in df_cleaned.select('Cycle').distinct().collect()])

cycles

In [ ]:
def assert_parent_bee_id(cycle):
    n = 3
    cycle_index = cycles.index(cycle)

    if  not cycle_index :
        return None

    min_cycle_index = 0

    if cycle_index > n:
        min_cycle_index = cycle_index - n

    parent_bee_id = random.randint(continuous_min_id_per_cycle[cycles[min_cycle_index]], continuous_min_id_per_cycle[cycles[cycle_index]] - 1)

    return parent_bee_id

In [ ]:
convertUDF = udf(lambda z: assert_parent_bee_id(z))

df_cleaned.withColumn("Parent Temp", convertUDF(col('Cycle'))).show()

In [ ]:
# from operator import add
def test2(a,b):
    return a+b

spark.sparkContext.parallelize([1, 2, 3, 4, 5]).fold(1, test2)

In [ ]:
def test1(a, b):
    #df_cleaned.filter(df_cleaned['Parent Temp'] == a[8])
    #print(a)
    print(b)

df_cleaned.rdd.reduce( test1 )